In [ ]:
import adversarial.carlini_wagner as cw
from adversarial.carlini_wagner import CWBuilder

params = {
    CWBuilder.USETQDM:"notebook",
    CWBuilder.MIN_IT:500,
    CWBuilder.LEARN_RATE:1e-4,
    CWBuilder.ADV_COEFF:0.125,
    CWBuilder.REG_COEFF:100,
    "eigs_num":40,
    CWBuilder.LOG:True}

target = 4
search_it = 5

adex_ours = cw.generate_adversarial_example(
    mesh=mesh, classifier=MODEL, target=target,
    search_iterations=search_it,
    lowband_perturbation=True, 
    adversarial_loss="carlini_wagner",
    similarity_loss="local_euclidean", 
    regularization="centroid", **params)

print("adversarial-attack: {}".format(adex_ours.is_successful.item()))
showperturbation(adex_ours)

params[CWBuilder.ADV_COEFF] = 0.001
adex_GeoA3 = cw.generate_adversarial_example(
    mesh=mesh, classifier=MODEL, target=target,
    search_iterations=search_it,
    lowband_perturbation=False, 
    adversarial_loss="log_softmax",
    similarity_loss="GeoA3", 
    regularization="none", **params)

print("adversarial-attack: {}".format(adex_GeoA3.is_successful.item()))
showperturbation(adex_GeoA3)

In [ ]:
import utils
from torch_geometric.data.data import Data
import shutil
from os import mkdir
from os.path import join, exists
import tqdm

def write_obj(pos, faces, file:str):
  utils.misc.check_data(pos=pos, faces=faces)
  pos = pos.cpu().detach().numpy()
  faces = faces.cpu().detach().numpy()

  file = file if file.split(".")[-1] == "obj" else file + ".obj"
  with open(file, 'w') as f:
      f.write("# OBJ file\n")
      for v in pos:
          f.write("v {} {} {}\n".format(v[0], v[1], v[2]))
          
      for face in faces:
          f.write("f")
          for i in face:
              f.write(" %d" % (i + 1))
          f.write("\n")

def GeoA3_experiments(search_iterations:int, build_params:dict, subjects:list, 
                      title:str="GeoA3_qualitative",replace_experiment=False):
    EXP_ROOT = "../experiments"
    experiment_root = join(EXP_ROOT, title)
    if exists(experiment_root):
        if replace_experiment:
          shutil.rmtree(experiment_root, ignore_errors=True)
        else:
          raise ValueError("experiment already exists!")
    mkdir(experiment_root)

    for sub_i in tqdm.tqdm_notebook(subjects):
        mesh = testdata[sub_i]
        mesh.to(DEVICE)
        _,I= MODEL(mesh.pos).detach().view(-1).sort()
        pred_y, y = I[-1].item(), mesh.y.item()
        if pred_y != y: raise ValueError("{} is different from {}!".format(pred_y,y))
        target = I[-4].item()
        adex_ours = cw.generate_adversarial_example(
            mesh=mesh, classifier=MODEL, target=target,
            search_iterations=search_it,
            lowband_perturbation=True, 
            adversarial_loss="carlini_wagner",
            similarity_loss="local_euclidean", 
            regularization="centroid", **params)
        print("adversarial-attack: {}".format(adex_ours.is_successful.item()))
        c = (adex_ours.perturbed_pos-adex_ours.pos).norm(p=2,dim=-1)
        visualize(adex_ours.perturbed_pos,faces=adex_ours.faces,intensity=c)

        adex_GeoA3 = cw.generate_adversarial_example(
            mesh=mesh, classifier=MODEL, target=target,
            search_iterations=search_it,
            lowband_perturbation=False, 
            adversarial_loss="log_softmax",
            similarity_loss="GeoA3", 
            regularization="none", **params)
        print("adversarial-attack: {}".format(adex_GeoA3.is_successful))

        fullname = lambda x:join(EXP_ROOT, title, x)
        write_obj(adex_ours.pos, adex_ours.faces, fullname("subject_{}_original".format(sub_i)))
        write_obj(adex_ours.perturbed_pos, adex_ours.faces, fullname("subject_{}_adv_ours".format(sub_i)))
        write_obj(adex_GeoA3.perturbed_pos, adex_GeoA3.faces, fullname("subject_{}_adv_GeoA3".format(sub_i)))

search_it = 10
subjects = [4,29,42,69,78]
params = {
    CWBuilder.USETQDM:None,
    CWBuilder.MIN_IT:500,
    CWBuilder.LEARN_RATE:1e-4,
    CWBuilder.ADV_COEFF:0.05,
    CWBuilder.REG_COEFF:100,
    "eigs_num":40,
    CWBuilder.LOG:False}

GeoA3_experiments(search_iterations=search_it, build_params=params, subjects=subjects,replace_experiment=True)